# Реализуем метод predict_proba для KNN

Ниже реализован класс KNeighborsClassifier, который для поиска ближайших соседей использует sklearn.neighbors.NearestNeighbors

Требуется реализовать метод predict_proba для вычисления ответа классификатора.

In [15]:
import numpy as np

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import NearestNeighbors


class KNeighborsClassifier(BaseEstimator, ClassifierMixin):
    '''
    Класс, который позволит нам изучить KNN
    '''
    def __init__(self, n_neighbors=5, weights='uniform', 
                 metric='minkowski', p=2):
        '''
        Инициализируем KNN с несколькими стандартными параметрами
        '''
        assert weights in ('uniform', 'distance')
        
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.metric = metric
        
        self.NearestNeighbors = NearestNeighbors(
            n_neighbors = n_neighbors,
            metric = self.metric)
    
    def fit(self, X, y):
        '''
        Используем sklearn.neighbors.NearestNeighbors 
        для запоминания обучающей выборки
        и последующего поиска соседей
        '''
        self.NearestNeighbors.fit(X)
        self.n_classes = len(np.unique(y))
        self.y = y
    def predict_proba(self, X, use_first_zero_distant_sample=True):
        '''
        Чтобы реализовать этот метод, 
        
        изучите работу sklearn.neighbors.NearestNeighbors
        '''
        # получим здесь расстояния до соседей distances и их метки
        distances, indexes = self.NearestNeighbors.kneighbors(X)
        
        probs = np.ones((X.shape[0], np.unique(y[indexes]).shape[0]))
        #print(probs)
        if self.weights == 'uniform':
            w = np.ones(distances.shape)
        else:
            # чтобы не делить на 0, 
            # добавим небольшую константу, например 1e-3
            w = 1/(distances + 1e-3)
        
        # реализуем вычисление предсказаний:
        # выбрав один объект, для каждого класса посчитаем
        # суммарный вес голосующих за него объектов
        # затем нормируем эти веса на их сумму
        # и вернем это как предсказание KNN
        
        w_class = 0
        
        norm = np.sum(w, axis=1)
        
        print("Classes: ", y[indexes])
        
        for class_n in range(np.unique(y[indexes]).shape[0]):
            for i in range(X.shape[0]):
                for j in range(distances.shape[1]):
                    if  class_n == y[indexes][i, j]:
                        w_class += w[i, j]
                probs[i, class_n] *= w_class/norm[i]
                w_class = 0
        print("Probs: ", probs)
        return probs

# Загрузим данные и обучим классификатор

In [16]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)

knn = KNeighborsClassifier(weights='distance')
knn.fit(X, y)
prediction = knn.predict_proba(X, )

Classes:  [[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 2]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 2 1]
 [1 1 1 1 1]
 [1 2 2 2 1]
 [1 1 1 1 1]
 [1 2 2 2 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [

Поскольку мы используем одну и ту же выборку для обучения и предсказания, ближайшим соседом любого объекта будет он же сам. В качестве упражнения предлагаю реализовать метод transform, который реализует получение предсказаний для обучающей выборки, но для каждого объекта не будет учитывать его самого.

Посмотрим, в каких объектах max(prediction) != 1:

In [11]:
inds = np.arange(len(prediction))[prediction.max(1) != 1]
print(inds)

# [ 56  68  70  72  77  83 106 110 119 123 127 133 134 138 146]

[ 56  68  70  72  77  83 106 110 119 123 127 133 134 138 146]


Несколько примеров, на которых можно проверить правильность реализованного метода:

In [12]:
for i in 1, 4, -1:
    print(inds[i], prediction[inds[i]])

# 68 [0.         0.99816311 0.00183689]
# 77 [0.         0.99527902 0.00472098]
# 146 [0.         0.00239145 0.99760855]

68 [0.         0.99816311 0.00183689]
77 [0.         0.99527902 0.00472098]
146 [0.         0.00239145 0.99760855]


# Ответы для формы

В форму требуется ввести max(prediction) для объекта. Если метод реализован верно, то ячейка ниже распечатает ответы, которые нужно ввести в форму

In [ ]:
for i in 56, 83, 127:
    print('{:.4f}'.format(max(prediction[i])))

In [ ]:
#Test
probs = np.ones((2, 3))

#print(probs)

w = np.array([
    [1.00000000e+03, 9.90099010e+00, 7.02141888e+00, 7.02141888e+00, 7.02141888e+00], 
    [1.00000000e+03, 7.02141888e+00, 5.74036070e+00, 5.74036070e+00, 4.45222500e+00]
])

y = np.array([
    [2, 0, 1, 2, 2],
    [1, 1, 2, 0, 0]
])

w_class = 0

norm = np.sum(w, axis = 1)

print("norm", norm)

for n_classes in range(3):
    for i in range(2):
        #print("Looking for class: ", n_classes)
        #print("Object: ", i)
        for j in range(5):
            #print(y[i,j])
            if n_classes == y[i, j]:
                #print("Found the class ", y[i,j], "!")
                w_class += w[i, j]
                #print("w_class: ", w_class)
        #print("norm: ", norm[i])
        probs[i, n_classes] *= w_class/norm[i]
        #print("probs: ", probs)
        w_class = 0

probs.sum(axis=1)
